In [2]:
try:
    from collections.abc import Mapping
except ImportError:
    from collections import Mapping

import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os
import numpy as np

c:\Users\sarta\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\envs\registration.py:280: UserWarning: WARN: cannot import name 'Mapping' from 'collections' (c:\Users\sarta\AppData\Local\Programs\Python\Python310\lib\collections\__init__.py)
  logger.warn(str(e))


AttributeError: partially initialized module 'gym' has no attribute 'core' (most likely due to a circular import)

In [1]:
seed = 42
logs = "logs"
model_path = "models"

# env = gym.make("ALE/Breakout-v5")
env = make_atari_env("ALE/Breakout-v5", n_envs=4, seed=seed)
env = VecFrameStack(env, n_stack=4)

NameError: name 'make_atari_env' is not defined